In [1]:
import cv2 
from tensorflow.keras.models import load_model
import mediapipe as mp
import tensorflow as tf 
import numpy as np

In [2]:
mp_face_mesh = mp.solutions.face_mesh

In [3]:
# get vgg_model 
vgg_model = load_model('vgg2_eye_cropped_100_400.h5')

In [4]:
def get_cropped_image_eye(image, mp_face_mesh):

    height, width, _ = image.shape

    face_mesh = mp_face_mesh.FaceMesh()
    results = face_mesh.process(image)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            p1, p2 = face_landmarks.landmark[70], face_landmarks.landmark[340]
            x1 = int(p1.x * width)
            y1 = int(p1.y * height)
            x2 = int(p2.x * width)
            y2 = int(p2.y * height)

            # Ensure y1 is smaller and y2 is larger
            if y1 > y2:
                y1, y2 = y2, y1

            top_left = (x1, y1)
            lower_right = (x2, y2)

        # crop the image
        cropped_image = tf.image.crop_to_bounding_box(image, y1, x1, y2-y1, x2-x1)

        # resize it to be (100, 400) for model training purpose
        cropped_image = tf.image.resize(cropped_image, (100,400)).numpy().astype('uint8')

        return cropped_image

In [5]:
def real_time_predict(image, model):
    image = np.expand_dims(image, axis=0)
    result = model.predict(image)
    return result[0][0] < 0.5

In [6]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    result = False
    success, image = cap.read()
    if not success:
        print('Ignoring empty camera frame.')
        continue

    height, width, _ = image.shape
    # Define the points for the rectangle - in this case, the entire frame
    pt1 = (0,0)
    pt2 = (width,height)

    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cropped_image = get_cropped_image_eye(image, mp_face_mesh)

    if cropped_image is not None:
        result = real_time_predict(cropped_image, vgg_model)
    else:
        print("No face detected")


    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Horizontally flip the image
    flipped_image = cv2.flip(image, 1)

    # if gazing is detected, add a rectangle to the entire frame 
    if result:
        flipped_image = cv2.rectangle(flipped_image, pt1, pt2, (0, 255, 0), 50)
        cv2.putText(flipped_image, 'YOU ARE LOOKING AT ME', (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3, cv2.LINE_AA)

    # Horizontally flip the image
    #   flipped_image = cv2.flip(image, 1)

    # Display the flipped image
    cv2.imshow('Image', flipped_image)

    if cv2.waitKey(5) & 0xFF == 27:
        break

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


No face detected
1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 69ms/step


KeyboardInterrupt: 

In [7]:
cap.release()
cv2.destroyAllWindows()